## hyperband tuner

This is (hopefully) the final model tuner in this project. It should produce models with an acceptable accuracy for production.

### Please ensure that the data augmentation with suitable augmentation parameters process has been run and there are *train.tfrecord*, *validation.tfrecord* and *test.tfrecord* files in *data/processed*. 

In [1]:
from os.path import join
import tensorflow as tf

from src.image_handling import decode_record

processed = join('data', 'processed')

features = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}
shape = (32, 32, 1)

train_dataset = decode_record(join(processed, 'train.tfrecord'), features, shape)
validation_dataset = decode_record(join(processed, 'validation.tfrecord'), features, shape)
test_dataset = decode_record(join(processed, 'test.tfrecord'), features, shape)

In [2]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop, SGD

def create_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(2**hp.Int('2**num_filter_0', 4, 6),
        (4,4) ,activation='relu', input_shape=(32, 32, 1)))

    for i in range(hp.Int('num_cnn_layers', 0, 3)):
        filter = 2**hp.Int('2**num_filter_' + str(i), 4, 7)
        model.add(layers.Conv2D(filter, (4,4), activation='relu', padding='same'))
        if hp.Boolean('pooling_' + str(i)):
            model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Flatten())
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        nodes = 2**hp.Int('2**num_nodes_' + str(i), 4, 7)
        model.add(layers.Dense(nodes, activation='relu'))
    
    model.add(layers.Dense(3, 'softmax'))

    optimizers = {
        'adam': Adam(),
        'sgd': SGD(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]),
            momentum=hp.Float('momentum', 0.6, 1, 0.1),
            nesterov=hp.Boolean('nesterov')),
        'rms': RMSprop(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]))
    }

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=optimizers[hp.Choice('optimizer', list(optimizers.keys()))],
        metrics=['acc'])

    return model

In [3]:
from datetime import datetime
from tensorboard.plugins.hparams import api
from kerastuner import Hyperband
from tensorflow import summary
from tensorflow.keras.callbacks import TensorBoard

class customTuner(Hyperband):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def on_trial_end(self, trial):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the hparams where the metrics of tensorboard are
        hparam_dir = join(trial_dir, trial.trial_id)
        hparams = trial.hyperparameters.values
        with summary.create_file_writer(hparam_dir).as_default():
            api.hparams(hparams, trial_id=trial.trial_id)

        print(datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
        
        super().on_trial_end(trial)

    def on_epoch_end(self, trial, model, epoch, logs):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the data where the metrics of tensorboard are
        hist_dir = join(trial_dir, trial.trial_id)
        with summary.create_file_writer(hist_dir).as_default():
            for layer in model.weights:
                summary.histogram(layer.name, data=layer, step=epoch)
        super().on_epoch_end(trial, model, epoch, logs)

In [4]:
from kerastuner import HyperParameters

hp=HyperParameters()
log_dir = join('logs', 'srp19')
timestamp = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

tuner = customTuner(
    create_model,
    hyperparameters=hp,
    objective='acc',
    max_epochs=200,
    factor=3,
    executions_per_trial=1,
    directory=log_dir,
    project_name=timestamp)

tuner.search_space_summary()

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [ EarlyStopping(monitor='loss', patience=3) ]

tuner.search(
    train_dataset,
    validation_data=validation_dataset,
    epochs=30,
    steps_per_epoch=100,
    validation_steps=100,
    verbose=0,
    callbacks=callbacks)

2019-11-21T18-59-49
2019-11-21T18-59-54
2019-11-21T18-59-58
2019-11-21T19-00-02
2019-11-21T19-00-07
2019-11-21T19-00-11
2019-11-21T19-00-16
2019-11-21T19-00-20
2019-11-21T19-00-26
2019-11-21T19-00-31
2019-11-21T19-00-35
2019-11-21T19-00-39
2019-11-21T19-00-42
2019-11-21T19-00-45
2019-11-21T19-00-49
2019-11-21T19-00-53
2019-11-21T19-00-56
2019-11-21T19-00-59
2019-11-21T19-01-03
2019-11-21T19-01-07
2019-11-21T19-01-11
2019-11-21T19-01-15
2019-11-21T19-01-18
2019-11-21T19-01-22
2019-11-21T19-01-27
2019-11-21T19-01-30
2019-11-21T19-01-33
2019-11-21T19-01-37
2019-11-21T19-01-41
2019-11-21T19-01-44
2019-11-21T19-01-48
2019-11-21T19-01-51
2019-11-21T19-01-56
2019-11-21T19-02-02
2019-11-21T19-02-08
2019-11-21T19-02-13
2019-11-21T19-02-19
2019-11-21T19-02-24
2019-11-21T19-02-29
2019-11-21T19-02-35
2019-11-21T19-02-47
2019-11-21T19-02-52
2019-11-21T19-03-00
2019-11-21T19-03-05
2019-11-21T19-03-15
2019-11-21T19-03-22
2019-11-21T19-03-38
2019-11-21T19-03-53
2019-11-21T19-04-04
2019-11-21T19-04-20


In [6]:
tuner.results_summary()

In [7]:
test_winner = tuner.get_best_models(num_models=10)

for winner in test_winner:
    print(winner.evaluate(test_dataset, steps=100, verbose=0))

[0.40828269101213666, 0.936875]
[0.31918077359441666, 0.9234375]
[0.4869032717659138, 0.9359375]
[0.38010222040116787, 0.909375]
[0.2098659366928041, 0.9434375]
[0.23279909185133874, 0.9440625]
[0.416561471009627, 0.915625]
[0.19408582380914596, 0.93875]
[0.21677776782773436, 0.92]
[0.21536825909046456, 0.9465625]
